<a href="https://colab.research.google.com/github/athi-raj/DiseasePredictor/blob/main/DiseasePredictior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DAY1

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/heart_disease_uci.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/heart_disease_uci.csv'

In [ ]:
df.head()

In [ ]:
print(df.columns)

In [ ]:
df.isnull().sum()

In [ ]:
numeric_cols=df.select_dtypes(include='number').columns
df[numeric_cols]=df[numeric_cols].fillna(df[numeric_cols].mean()) # checks the missing of numeric values

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df[numeric_cols].hist(figsize=(10,15))
plt.tight_layout()
plt.show()

In [ ]:
sns.heatmap(df[numeric_cols].corr(),annot=True,cmap='coolwarm')
plt.title('Numeric Feature Corelations')
plt.show()

DAY 2 NODEL TRAINING

In [ ]:
cat_cols = df.select_dtypes(include='object').columns.tolist()
if 'num' in cat_cols:
  cat_cols.remove('num')

In [ ]:
x = df.drop('num',axis=1)
y = (df['num'] > 0).astype(int)
cat_cols = x.select_dtypes(include='object').columns.tolist()

NameError: name 'df' is not defined

In [ ]:
x = pd.get_dummies(x, columns=cat_cols)
print("Final feature columns:", x.columns)

DAY 3 Feature engg

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_model=LogisticRegression()
lr_model.fit(x_train_scaled,y_train)

In [ ]:
from sklearn.metrics import accuracy_score , classification_report

In [ ]:
y_pred_lr=lr_model.predict(x_test_scaled)
print("LogisticRegression Accuracy:",accuracy_score(y_test,y_pred_lr))
print(classification_report(y_test,y_pred_lr))

DAY 4

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm=confusion_matrix(y_test,y_pred_lr)
sns.heatmap(cm,annot=True,fmt='d',cmap='Blues')
plt.title('Confusion matric (Logistic regression)')
plt.show()

random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_scaled, y_train)
y_pred_rf = rf_model. predict(x_test_scaled)

In [ ]:
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

In [ ]:
feat_imp = pd.Series(rf_model.feature_importances_, index=x.columns)
feat_imp.nlargest(10).plot(kind='barh')
plt.title('Random Forest Feature Importance')
plt.show()

Save the model

In [ ]:
import joblib
joblib.dump(rf_model, 'heart_rf_model.pkl')

In [ ]:
joblib.dump(scaler, 'heart_scaler.pkl')

In [ ]:
sample = x.head(1)
sample.to_csv('Heart_user_template.csv', index=False)
print("User Template saved as 'Heart_user_terplate.csv' ")

Day 5


In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
import joblib
import pandas as pd

user_df=pd.read_csv('heart_dataset.csv')
num_col = df.select_dtypes(include='number').columns.tolist()
cat_col = df.select_dtypes(include='object').columns.tolist()
bool_col = df.select_dtypes(include='bool').columns.tolist()


num_col = [col for col in num_col if col in user_df.columns]
cat_col = [col for col in cat_col if col in user_df.columns]
bool_col = [col for col in bool_col if col in user_df.columns]

user_df[num_col]=user_df[num_col].fillna(user_df[num_col].mean())

for col in cat_col:
  user_df[col]=user_df[col].fillna('unknown')

for col in bool_col:
  user_df[col]=user_df[col].astype(int)


user_df_encoded = pd.get_dummies(user_df,columns=cat_col)

user_df_encoded = user_df_encoded.reindex(columns=x.columns, fill_value=0)

scaler = joblib.load("heart_scaler.pkl")
user_df_scaled = scaler.transform(user_df_encoded)

model = joblib.load("heart_rf_model.pkl")
pred = model.predict(user_df_scaled)
user_df['Heart_Disease_Pediction']=pred
print(user_df)

   age  trestbps  chol  fbs         restecg  thalch  exang  oldpeak  \
0   58       130   220    1          normal     150  False      1.4   
1   67       160   276    0  lv hypertrophy     108   True      1.5   
2   42       120   230    0          normal     170  False      1.0   
3   50       130   210    0  lv hypertrophy     158  False      0.8   
4   45       114   230    0          normal     165  False      1.1   

         slope  ca               thal  sex_Female  sex_Male  cp_asymptomatic  \
0         flat   0       fixed defect           0         1                0   
1         flat   3             normal           0         1                1   
2    upsloping   0  reversable defect           1         0                0   
3         flat   0             normal           0         1                0   
4  downsloping   0             normal           1         0                0   

   cp_atypical angina  cp_non-anginal  cp_typical angina  \
0                   0           